In [33]:
# !python -m spacy download en_core_web_sm
import re

import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

In [30]:
sample_data= pd.read_csv('sample_service_contracts.csv')
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0.1  100 non-null    int64  
 1   Unnamed: 0    100 non-null    int64  
 2   principal     100 non-null    object 
 3   agent         100 non-null    object 
 4   amount        100 non-null    float64
 5   service_type  100 non-null    object 
 6   confidence    100 non-null    float64
 7   PDF_ID        100 non-null    object 
 8   Folder_Name   100 non-null    object 
 9   surya_ocr     100 non-null    object 
 10  Purpose       74 non-null     object 
dtypes: float64(2), int64(2), object(7)
memory usage: 8.7+ KB


In [53]:
sample_data.head(10)

,Unnamed: 0.1,Unnamed: 0,principal,agent,amount,service_type,confidence,PDF_ID,Folder_Name,surya_ocr,Purpose
0,0,177,Hawkeye Community College,Iowa Workforce Development,0.0,Employment of Regional Iowa Works Director,0.95,M503303,Economic Development_OCR,FILED\nMichael A. Mauro\nOFFICE USE OF\n28E\nS...,the purpose of this agreement is: to establish...
1,1,84,City of Lawler,Chickasaw County,3292.5,Law enforcement and police protection,0.98,M507033,Police Protection_OCR,FILED\nMatt Schultz\nOFFICE USE OF\n28E\nSecre...,the purpose of this agreement is: 28e law enfo...
2,2,42,City of Wall Lake,City of Lake View,96360.0,Law enforcement and police coverage,0.90,M506822,Police Protection_OCR,FILED\nMatt Schultz\nOFFICE USE OF\n28E\nSecre...,the purpose of this agreement is: the city of ...
3,3,89,CITY OF LAWLER,CHICKASAW COUNTY,14795.0,General law enforcement services,0.99,M505256,Police Protection_OCR,FILED\nMatt Schultz\nOFFICE USE OF\n28E\nSecre...,the purpose of this agreement is: 28e agreement
4,4,151,City of Pacific Junction,Mills County,0.0,Communications center and emergency dispatch s...,0.95,M037894,Emergency Management_OCR,00337\n28E AGREEMENT\nMILLS COUNTY COMMUNICATI...,NaN
5,5,6,City of Maurice,Sioux County,17702.0,Law enforcement services,1.00,M506133,Police Protection_OCR,FILED\nMatt Schultz\nOFFICE USE ON\n28E\nSecre...,the purpose of this agreement is: renewal of c...
6,6,101,City of Forest City,Forest City Ambulance Service,0.0,Funding of paramedic ambulance services,0.95,M507329,Health_OCR,FILED\nMatt Schultz\nOFFICE USE OF\n28E\nSecre...,the purpose of this agreement is: funding of p...
7,7,222,Iowa Valley Community College District,"Lennox Industries, Inc.",0.0,Industrial New Jobs Training,0.90,L005128,Economic Development_OCR,03658 -\nSECI\nCar 30 - 10 box 19 193\nOV 1 6 ...,NaN
8,8,87,City of Maurice,Sioux County,17138.0,Law enforcement services,0.99,M505476,Police Protection_OCR,FILED\nMatt Schultz\nOFFICE USE ON\n28E\nSecre...,the purpose of this agreement is: amendment of...
9,9,194,Iowa Workforce Development,Hawkeye Community College,0.0,Hiring and employment of Operations Director,0.95,M503301,Economic Development_OCR,FILED\nMichael A. Mauro\nOFFICE USE ON\n28E\nS...,the purpose of this agreement is: to establish...


In [ ]:
test = sample_data['principal'].value_counts()

# Print all items (ensures all rows are shown, not truncated)
print(test.to_string())
identifiers = ['city of', 'county', 'iowa', 'town', 'college']

principal
Iowa Workforce Development                                                          15
City of Des Moines                                                                   5
City of Maurice                                                                      4
Hawkeye Community College                                                            2
City of Collins                                                                      2
City of Ferguson                                                                     2
City of Zearing                                                                      2
Jasper County, Iowa                                                                  2
City of Dysart                                                                       2
City of Boyden                                                                       2
Iowa Valley Community College District                                               2
City of Carlisle                 

In [44]:
sample_ocr = sample_data.at[1, 'surya_ocr']

doc = nlp(sample_ocr)
matches = []
seen = set()
for ent in doc.ents:
    entity_pattern = r"(?:City|County|Town|Village|Township)\s+of\s+[\w\s]+"
    found_matches = re.findall(entity_pattern, ent.text, re.IGNORECASE)
    for match in found_matches:
        if match not in seen:
            matches.append(match)
            seen.add(match)
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print(f"end state: {matches}")

Matt Schultz 6 18 PERSON
28E 33 36 CARDINAL
State 50 55 ORG
5/14/2014 11:25:25 AM
PLEASE 88 116 TIME
1 176 177 CARDINAL
1 320 321 CARDINAL
CHickasaw County 322 338 GPE
2
City 362 368 PERCENT
Lawler
City 372 383 GPE
3 400 401 CARDINAL
4 408 409 CARDINAL
5 416 417 CARDINAL
Sowa
Item 2 443 454 ORG
Public Service 468 482 ORG
110 514 517 CARDINAL
only one 543 551 CARDINAL
3 619 620 CARDINAL
28E Law Enforcement Services Contract
Item 679 721 ORG
Item 5 836 842 LAW
28E 1093 1096 CARDINAL
two 1150 1153 CARDINAL
Phone 1399 1404 ORG
641-394-2100 1405 1417 PRODUCT
5 1418 1419 CARDINAL
Chickasaw County 1497 1513 GPE
8 1514 1515 CARDINAL
IA 1543 1545 GPE
NA 1574 1576 ORG
8 1622 1623 CARDINAL
East Prospect PO Box 1624 1644 ORG
311 1645 1648 CARDINAL
New Hampton 1649 1660 PERSON
IA 1662 1664 GPE
Todd W. Miller 1828 1842 PERSON
Chickasaw County Sheriff 1844 1868 GPE
N. Chestnut Avenue 1874 1892 PERSON
New Hampton 1894 1905 GPE
IA 1907 1909 GPE
50659 1910 1915 DATE
394 3121 1920 1928 DATE
24 hours 1930

In [29]:
import re

sample_ocr = sample_data.at[3, 'surya_ocr']
# Expanded identifiers based on common patterns in principal names
identifiers = [
    'city of', 'county', 'iowa', 'town', 'college', 
    'township', 'school district', 'department', 'state',
    'community college', 'community', 'sheriff', 'board'
]
doc = nlp(sample_ocr)
matches = []
seen = set()  # Use set for O(1) lookup performance
money_entities = []  # Collect MONEY entities

# Create regex pattern that matches any identifier (case-insensitive)
# Use word boundaries for single words to avoid partial matches
identifier_pattern = '|'.join(
    r'\b' + re.escape(identifier) + r'\b' if ' ' not in identifier 
    else re.escape(identifier) 
    for identifier in identifiers
)
pattern = re.compile(identifier_pattern, re.IGNORECASE)

for ent in doc.ents:
    if ent.label_ in ['GPE']:
        # Check if any identifier is found in ent.text
        if pattern.search(ent.text):
            # Normalize the text (strip, remove extra spaces) before checking
            normalized_text = ' '.join(ent.text.split())
            if normalized_text not in seen:
                matches.append(normalized_text)
                seen.add(normalized_text)
    elif ent.label_ == 'MONEY':
        # Collect money entities
        money_entities.append({
            'text': ent.text,
            'start': ent.start_char,
            'end': ent.end_char
        })
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

print(f"end state: {matches}")

# If MONEY entities found, analyze transactions using matches and money
if money_entities:
    print(f"\n=== TRANSACTION ANALYSIS ===")
    print(f"Found {len(money_entities)} money entity/entities: {[m['text'] for m in money_entities]}")
    print(f"Found {len(matches)} potential principal(s): {matches}")
    
    # Payment patterns to search for in the window
    payment_patterns = [
        # X shall pay Y / X agrees to pay Y
        r"(?P<payer>[\w\s]+?)\s+(?:shall|agrees to|will)\s+pay\s+(?:to\s+)?(?:the\s+)?(?P<payee>[\w\s]+?)(?:\s+(?:a|an|the)\s+(?:sum|amount))",
        # Payment from X to Y
        r"payment\s+(?:shall be made\s+)?(?:from\s+)?(?:the\s+)?(?P<payer>[\w\s]+?)\s+to\s+(?:the\s+)?(?P<payee>[\w\s]+)",
        # X pays Y
        r"(?P<payer>[\w\s]+?)\s+pays?\s+(?:to\s+)?(?:the\s+)?(?P<payee>[\w\s]+?)(?:\s+(?:a|an|the)\s+(?:sum|amount|fee))",
        # Y shall be paid by X
        r"(?P<payee>[\w\s]+?)\s+shall be paid by\s+(?:the\s+)?(?P<payer>[\w\s]+)",
    ]
    
    # Compile patterns for case-insensitive matching
    compiled_patterns = [re.compile(pattern, re.IGNORECASE) for pattern in payment_patterns]
    
    # Analyze proximity between matches and money entities
    # Find entities near money amounts (within a window of characters)
    window_size = 500  # characters before/after money entity
    
    for money in money_entities:
        print(f"\n{'='*60}")
        print(f"Money: {money['text']} (position {money['start']}-{money['end']})")
        
        # Extract window of text around the money entity
        window_start = max(0, money['start'] - window_size)
        window_end = min(len(sample_ocr), money['end'] + window_size)
        window_text = sample_ocr[window_start:window_end]
        
        print(f"\nSearching for payment patterns in window (chars {window_start}-{window_end})...")
        
        # Search for payment patterns in the window
        found_patterns = []
        for i, pattern in enumerate(compiled_patterns):
            for match in pattern.finditer(window_text):
                payer = match.group('payer').strip()
                payee = match.group('payee').strip()
                # Adjust match position to absolute position in document
                abs_start = window_start + match.start()
                abs_end = window_start + match.end()
                
                found_patterns.append({
                    'pattern_num': i + 1,
                    'payer': payer,
                    'payee': payee,
                    'match_text': match.group(0),
                    'position': (abs_start, abs_end)
                })
        
        if found_patterns:
            print(f"  Found {len(found_patterns)} payment pattern(s):")
            for pattern_match in found_patterns:
                print(f"    Pattern {pattern_match['pattern_num']}:")
                print(f"      Payer: {pattern_match['payer']}")
                print(f"      Payee: {pattern_match['payee']}")
                print(f"      Match: {pattern_match['match_text'][:100]}...")
                print(f"      Position: {pattern_match['position']}")
        else:
            print(f"  No payment patterns found in window")
        
        # Also check for nearby entities
        nearby_entities = []
        for match in matches:
            # Find the position of this match in the original text
            match_pos = sample_ocr.find(match)
            if match_pos != -1:
                distance = abs(match_pos - money['start'])
                if distance <= window_size:
                    nearby_entities.append({
                        'entity': match,
                        'position': match_pos,
                        'distance': distance
                    })
        
        if nearby_entities:
            print(f"\n  Nearby entities (within {window_size} chars):")
            for entity_info in sorted(nearby_entities, key=lambda x: x['distance']):
                print(f"    - {entity_info['entity']} (distance: {entity_info['distance']} chars)")
        else:
            print(f"\n  No nearby entities found within {window_size} characters")


Matt Schultz 6 18 PERSON
28E 33 36 CARDINAL
State 50 55 ORG
4:22:44 PM 98 108 TIME
1 175 176 CARDINAL
LAWLER 329 335 GPE
2 357 358 CARDINAL
3 399 400 CARDINAL
4 407 408 CARDINAL
5 415 416 CARDINAL
Sowa
Item 2 442 453 ORG
Public Service 467 481 ORG
110 513 516 CARDINAL
only one 542 550 CARDINAL
3 618 619 CARDINAL
28e 677 680 CARDINAL
Item 5 810 816 LAW
28E 1067 1070 CARDINAL
two 1124 1127 CARDINAL
Title County 1312 1324 GPE
Phone 1387 1392 ORG
641-394-2100 1393 1405 PRODUCT
Marty Larsen 1440 1452 PERSON
Chickusaw County Sheriff 1454 1478 GPE
116 1480 1483 CARDINAL
North Chestnut 1484 1498 GPE
New Hampton 1500 1511 GPE
IA 1513 1515 GPE
AGREEMENT 1566 1575 ORG
Ma 1 1611 1615 ORG
2012 1625 1629 DATE
Chickasaw County 1646 1662 GPE
the "County 1691 1702 GPE
Lawler 1721 1727 GPE
Iowa 1729 1733 GPE
1 1774 1775 CARDINAL
The County 1800 1810 GPE
the Chickasaw County Sheriff 1833 1861 ORG
Law Enforcement 1889 1904 PERSON
Sheriff 1979 1986 FAC
2 2074 2075 CARDINAL
Chapter 28E, Code of Iowa 2139 21

In [32]:
part_1_data = pd.read_csv('contracts_part1_extracted.csv')
part_1_data.head(10)

,PDF_ID,num_parties,num_financial_chunks,num_money_references,parties_list,parties_json,amounts_found,max_amount,min_amount,financial_chunks_json,money_references_json
0,M503303,7,3,0,Iowa Workforce Development | Hawkeye Community...,"[{""name"": ""Iowa Workforce Development"", ""extra...",NaN,0.00,0.0,"[{""chunk_text"": ""or relations policies of both...",[]
1,M507033,4,2,2,"Chickasaw County | City of Lawler | ""Enter ""Ot...","[{""name"": ""Chickasaw County"", ""extraction_meth...","$3,292.50, $7.50",3292.50,7.5,"[{""chunk_text"": ""contract. That question is ex...","[{""amount_str"": ""$7.50"", ""amount_value"": 7.5, ..."
2,M506822,3,1,4,"Lake View | Wall Lake | 'Enter ""Other"" if","[{""name"": ""Lake View"", ""extraction_method"": ""f...","$4,095.00, $3,935.00, $46.80, $45.00",4095.00,45.0,"[{""chunk_text"": ""erk Title City Clerk Email Iv...","[{""amount_str"": ""$3,935"", ""amount_value"": 3935..."
3,M505256,4,1,1,"CITY OF LAWLER | Chickasaw COUNTY | ""Enter ""Ot...","[{""name"": ""CITY OF LAWLER"", ""extraction_method...","$14,795.00",14795.00,14795.0,"[{""chunk_text"": ""on or property caused by the ...","[{""amount_str"": ""$14,795.00"", ""amount_value"": ..."
4,M037894,2,2,1,Attach additional page(s) II no | The County,"[{""name"": ""Attach additional page(s) II no"", ""...",$8.00,8.00,8.0,"[{""chunk_text"": ""- shall be provided by the Co...","[{""amount_str"": ""$8.00"", ""amount_value"": 8.0, ..."
5,M506133,3,1,1,"Sioux County | City of Maurice | ""Enter ""Other...","[{""name"": ""Sioux County"", ""extraction_method"":...","$17,702.00",17702.00,17702.0,"[{""chunk_text"": ""l 3) Maurice agrees to pay Co...","[{""amount_str"": ""$17,702."", ""amount_value"": 17..."
6,M507329,3,2,4,City of Forest City | Forest City Ambulance Se...,"[{""name"": ""City of Forest City"", ""extraction_m...","$5.00, $2.25, $2.00, $1.00",5.00,1.0,"[{""chunk_text"": ""se of this Agreement is 635 H...","[{""amount_str"": ""$2.25"", ""amount_value"": 2.25,..."
7,L005128,2,4,3,IOWA VALLEY COMMUNITY COLLEGE DISTRICT (MERGED...,"[{""name"": ""IOWA VALLEY COMMUNITY COLLEGE DISTR...","$64.43, $23.50, $23.50",64.43,23.5,"[{""chunk_text"": ""03658 - SECI Car 30 - 10 box ...","[{""amount_str"": ""$23.5"", ""amount_value"": 23.5,..."
8,M505476,3,1,1,"Sioux County | City of Maurice | ""Enter ""Other...","[{""name"": ""Sioux County"", ""extraction_method"":...","$17,138.00",17138.00,17138.0,"[{""chunk_text"": ""1 3) Maurice agrees to pay Co...","[{""amount_str"": ""$17,138."", ""amount_value"": 17..."
9,M503301,7,3,0,Iowa Workforce Development | Hawkeye Community...,"[{""name"": ""Iowa Workforce Development"", ""extra...",NaN,0.00,0.0,"[{""chunk_text"": ""nd labor relations policies o...",[]
